<a href="https://colab.research.google.com/github/fabian-stettler/AISO/blob/main/00_search_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise Search Algorithms #

## Introduction
The swiss railway system publishes various datasets in an open data initiative (https://data.sbb.ch). You are going to write your own search algorithms and test them on the swiss railway system. This will help you to understand how the different search strategies work and how they perform on a real-world problem.

The json-file `linie-mit-betriebspunkten.json` contains the data about the different railway lines of the swiss railway system. Later on, we are going to import the location of the different hubs and how they are connected. But for now, we just have a look at an image generated by this data set:

![sbb-map](https://github.com/iaherzog/search/blob/main/sbb-map.jpg?raw=true)

You can see the train station of Rotkreuz in the center of the image and how Rotkreuz is connnected to other stations with different train lines. With this knowledge, an agent will find a path from one train station to another.

> By the way, what kind of agent-type will you use, if the agent wants to find the shortest path between two stations?

## Creating a graph

Before we start to implement the different search algorithms, we start to define some auxiliary classes.

> Are we dealing here with a **graph-based** or **tree-based** search problem? And why?

It is useful to define a graph class that can be used later on. This class contains some basic properties and functions. The class implementation can be seen below. Carefully read the code and try to understand its functions.

In [1]:
class Graph:

    """A graph connects nodes by edges.  Each edge can also
    have a length associated with it.  The constructor call is something like:
        g = Graph({'A': {'B': 1, 'C': 2})
    this makes a graph with 3 nodes, A, B, and C, with an edge of length 1 from
    A to B,  and an edge of length 2 from A to C.  You can also do:
        g = Graph({'A': {'B': 1, 'C': 2}, directed=False)
    This makes an undirected graph, so inverse links are also added. The graph
    stays undirected; if you add more edges with g.connect('B', 'C', 3), then
    inverse link is also added.  You can use g.get_nodes() to get a list of nodes,
    g.get_edges('A') to get a dict of edges out of A, and g.get_distance('A', 'B') to get the
    length of the edge from A to B. """

    def __init__(self, graph_dict=None, directed=True):
        self.graph_dict = graph_dict or {}
        self.directed = directed
        if not directed:
            self.make_undirected()

    def make_undirected(self):
        """Make a digraph into an undirected graph by adding symmetric edges."""
        for a in list(self.graph_dict.keys()):
            for (b, dist) in self.graph_dict[a].items():
                self.add_connection(b, a, dist)

    def connect(self, A, B, distance=1):
        """Add a link from A and B of given distance, and also add the inverse
        link if the graph is undirected."""
        self.add_connection(A, B, distance)
        if not self.directed:
            self.add_connection(B, A, distance)

    def add_connection(self, A, B, distance):
        """Add a link from A to B of given distance, in one direction only."""
        self.graph_dict.setdefault(A, {})[B] = distance

    def get_edges(self, a):
        return self.graph_dict.setdefault(a, {})

    def get_distance(self, a, b):
        links = self.graph_dict.setdefault(a, {})
        return links.get(b)

    def get_nodes(self):
        """Return a list of nodes in the graph."""
        s1 = set([k for k in self.graph_dict.keys()])
        s2 = set([k2 for v in self.graph_dict.values() for k2, v2 in v.items()])
        nodes = s1.union(s2)
        return list(nodes)



Based on this code, an undirected graph class can be created, where every edge goes in both directions:

In [2]:
def UndirectedGraph(graph_dict=None):
    """Build a Graph where every edge (including future ones) goes both ways."""
    return Graph(graph_dict = graph_dict, directed=False)

To illustrate the use of the `Graph` and the `UndirectedGraph` class, we define our textbook problem. The `romania_graph` is an undirected graph which is initialized with the different cities and the connections to the other cities:

In [3]:
romania_graph = UndirectedGraph(dict(
    Arad=dict(Zerind=75, Sibiu=140, Timisoara=118),
    Bucharest=dict(Urziceni=85, Pitesti=101, Giurgiu=90, Fagaras=211),
    Craiova=dict(Drobeta=120, Rimnicu=146, Pitesti=138),
    Drobeta=dict(Mehadia=75),
    Eforie=dict(Hirsova=86),
    Fagaras=dict(Sibiu=99),
    Hirsova=dict(Urziceni=98),
    Iasi=dict(Vaslui=92, Neamt=87),
    Lugoj=dict(Timisoara=111, Mehadia=70),
    Oradea=dict(Zerind=71, Sibiu=151),
    Pitesti=dict(Rimnicu=97),
    Rimnicu=dict(Sibiu=80),
    Urziceni=dict(Vaslui=142)))

Let's see how we can use this map:

First, how can you get a list of the children (connected nodes) from a given city, let's say from Arad?

In [4]:
romania_graph.get_edges('Arad')

{'Zerind': 75, 'Sibiu': 140, 'Timisoara': 118}

And how can you get the distance between Arad and Zerind?

In [5]:
romania_graph.get_distance('Arad', 'Zerind')

75

## Formulating a Search Problem

Do you remember, what needs to be defined, in order to generate a search problem? We already have the *graph*, but to generate a *problem*, the following things need to be defined:

- States
    - Initial state
    - Goal state
- Actions
- Goal test
- Path costs

Luckily, there is already a `GraphProblem` class implemented:

In [6]:
class GraphProblem():

    """The problem of searching a graph from one node to another."""

    def __init__(self, initial, goal, graph):
        self.initial = initial
        self.goal = goal
        self.graph = graph

    def goal_test(self, state):
        """Return True if the state is the goal. """
        return state == self.goal

    def get_actions_from(self, A):
        """The actions at a graph node are just its neighbors."""
        return list(self.graph.get_edges(A).keys())

    def get_path_cost(self, A, B):
        return self.graph.get_distance(A, B)

    def get_solution():
      pass


Let's try to generate a problem instance based on our `romania_graph'

Create a problem instance with the initial state 'Arad' and goal state 'Bucharest':

In [7]:
start = 'Arad'
goal = 'Bucharest'
problem = GraphProblem(start, goal, romania_graph)

Test, if 'Bucharest' is indeed the goal state, and that 'Arad' is not the goal state.

In [8]:
assert problem.goal_test('Bucharest')
assert not problem.goal_test('Arad')


Which actions are available in Arad?

In [9]:
problem.get_actions_from('Arad')

['Zerind', 'Sibiu', 'Timisoara']

What is the cost of going from Arad to Zerind? (Doing the action 'Zerind' in 'Arad')

In [10]:
problem.get_path_cost('Arad', 'Zerind')

75

## The node class
The next step is to define the node class. What functions should this class provide? How would you initialize a node class?

If you have carefully listened to the lecture, you should have become familiar with the concept of a node. Have a look at the slides, it will help you to write the code.

Use the following template and try to implement the constructor and the different methods.

In [22]:
class Node:
    """A node in a search tree."""

    def __init__(self, state, parent=None, action=None, path_cost=0):
        """Create a search tree Node, derived from a parent by an action."""
        self.parent = parent
        self.node_producing_action = action
        self.path_cost = path_cost
        self.state = state


    def create_child_node(self, problem, action):
        # create a child node that is generated by the given action
        possible_actions = problem.get_actions_from(self.state)
        added_path_cost = problem.get_path_cost(self.state, action)

        #action is called the same as state
        new_node = Node(action, self, action, self.path_cost + added_path_cost)
        return new_node

    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        # create a list with nodes that can be expanded form this node
        # first get all actions and then create the successor nodes
        possible_actions = list(problem.get_actions_from(self.state))
        actual_expansion_nodes = []
        for action in possible_actions:
          actual_expansion_nodes.append(self.create_child_node(problem, action))

        return actual_expansion_nodes

    def get_path_from_root(self):
        """Return a list of nodes forming the path from the root to this node."""
        # Hint: use self.parent to create the list of nodes back to the root and reverse it
        temp_node = self
        path_to_root= []
        while temp_node.parent != None:
          path_to_root.append(temp_node)
          temp_node = self.parent

        #append root node
        path_to_root.append(self)

        return list(reversed(path_to_root))

    def get_solution(self):
        """Return the sequence of actions to go from the root to this node."""
        # Hint: use the self.get_path_from_root() function
        path_to_this_node = self.get_path_from_root()
        action_chain = []
        for node in path_to_this_node:
          action_chain.append(node.node_producing_action)
        return action_chain





You can test your code here:

In [24]:
# create first node of the problem
initial_node = Node(problem.initial)
print('initial node : ' + initial_node.state)

print('children of initial node: ')
children = initial_node.expand(problem)
for child in children:
    print('\t' + child.state + " with cost = " + str(child.path_cost))

# lets create the child node by doing the action 'Sibiu'
action = 'Sibiu'
next_node = initial_node.create_child_node(problem, action)
print('Next node created by action Sibiu leads to: ' + next_node.state)



initial node : Arad
children of initial node: 
	Zerind with cost = 75
	Sibiu with cost = 140
	Timisoara with cost = 118
Next node created by action Sibiu leads to: Sibiu
